In [4]:
import pandas as pd
import numpy as np

import InfoTrayectosModule as Tracks
from DecoderModule import Decoder
import InfoStationsModule as Stations


# Download all files from web
Using updateFiles(), we compare the list of links with ../Raw/DownloadedFiles.txt. Not listed links are downloaded and uncompress in: ../Raw/(Stations/Tracks)

In [4]:
import LocalAutomation as aut


aut.updateFiles()

/getattachment/8bb73c41-eab0-4e6a-ac92-80c8c68aacc2/201704_Usage_Bicimad.aspx is already saved
/getattachment/11054216-35d1-4003-b76b-8421c4a46eb4/201705_Usage_Bicimad.aspx is already saved
/getattachment/c3a71f16-490a-42f9-b02e-384e08ceac2a/201706_Usage_Bicimad.aspx is already saved
/getattachment/360dcfb0-78a2-49ae-8454-49e97530c1e4/201707_Usage_Bicimad.aspx is already saved
/getattachment/90cba892-9dc8-4120-b3e1-ca99d783bb5d/201708_Usage_Bicimad.aspx is already saved
/getattachment/4ec0d958-b441-4da0-8730-b9b5d763cdf9/201709_Usage_Bicimad.aspx is already saved
/getattachment/b237a0fc-2785-4930-8fa2-11225d839ff9/201710_Usage_Bicimad.aspx is already saved
/getattachment/dfb84318-8479-4bf1-93a8-957e7f7951e5/201711_Usage_Bicimad.aspx is already saved
/getattachment/b3df7d48-90a7-4817-a800-e84e5bfd3118/201712_Usage_Bicimad.aspx is already saved
/getattachment/4a6fa0af-d067-4d1a-84f4-a65fc4579640/201801_Usage_Bicimad.aspx is already saved
/getattachment/830d666c-62ad-4378-b3a8-2a47ea62e8f

# Stations

There is a problem with 2018-07 station file. Erase it!

## From json to pickle
Take all files from ../Raw/Stations and transform them to pickle creating a table. 

In [5]:
filePath= "../Raw/Stations"

Stations.fromJSONtoPickle(filePath)

Finished!


## Concatenation of pickles files
Concatenate all pkl files from Station. In order not overload memory it uses yield. It is also convinient, to choose the variable to use.<br>
As Stations.concatenateAll(filePath)['DockBikes']

In [6]:
filePath= "../DataFrames/Stations/"
data=Stations.concatenateAll(filePath)

1
['201907.pkl', '201908.pkl', '201909.pkl', '201910.pkl', '201911.pkl', '201912.pkl', '202001.pkl', '202002.pkl', '202003.pkl', '202004.pkl', '202005.pkl', '202006_stations.pkl', 'Bicimad_Estacions_201808.pkl', 'Bicimad_Estacions_201809.pkl', 'Bicimad_Estacions_201810.pkl', 'Bicimad_Stations_201811.pkl', 'Bicimad_Stations_201812.pkl', 'Bicimad_Stations_201901.pkl', 'Bicimad_Stations_201902.pkl', 'Bicimad_Stations_201903.pkl', 'Bicimad_Stations_201904.pkl', 'Bicimad_Stations_201905.pkl', 'Bicimad_Stations_201906.pkl']


## Decoder creation

Index file from web, to check Ids, names, coordinates etc... is outdated. Ids do not change, but as the stations may change over time, it is important to have an updated list. With Station file is possible to obtain it. I dont think is necessary to keep it it in the main table of Stations, because is repeated information.

It is not necessary to repeat this operation in a daily routine. Everytime a new station file is available would be the best. With savedecoderList() all files are
concatenate, and obtain the info of ID, Address, Longitude and Latitude that is saved in ../DataFrame/DatosEstaticos.txt



In [3]:
Stations.SaveDecoderList()

1
['201907.pkl', '201908.pkl', '201909.pkl', '201910.pkl', '201911.pkl', '201912.pkl', '202001.pkl', '202002.pkl', '202003.pkl', '202004.pkl', '202005.pkl', '202006_stations.pkl', 'Bicimad_Estacions_201808.pkl', 'Bicimad_Estacions_201809.pkl', 'Bicimad_Estacions_201810.pkl', 'Bicimad_Stations_201811.pkl', 'Bicimad_Stations_201812.pkl', 'Bicimad_Stations_201901.pkl', 'Bicimad_Stations_201902.pkl', 'Bicimad_Stations_201903.pkl', 'Bicimad_Stations_201904.pkl', 'Bicimad_Stations_201905.pkl', 'Bicimad_Stations_201906.pkl']
Concatenado


0

## Reduced data

Functions to treat data:

ChooseRange: User choose a value to show in a daterange. This function ALWAYS must be called<br>

ChooseInterval: Set the accuracy of the chosen range

ChooseComparator: In case user wanted to compare data: days, weekday, business...




In [18]:
def ChooseRange(df, StartDate, EndDate, Value ):
    c0 = df.index.to_series().between(StartDate, EndDate)
    return df[c0][Value]
def ChooseAccuracy(df, Accuracy):
    return df.resample(Accuracy).median()

def ChooseComparison(df, Comparator, Interval=''):
        Decisions={'Y': df.index.year,
                  'M': df.index.month,
                  'd': df.index.day,
                  'dow':df.index.dayofweek,
                  'h': df.index.hour,
                  'bd': df.index.dayofweek<5
        }
        if Comparator=='':
            return df.groupby([Decisions[Comparator]]).median()
        else:
            
            return df.groupby([Decisions[Comparator], Decisions[Interval]]).median()

def Clusterization(df, cluster, op, Decode):
    df = df.rename(columns = lambda x: Decode.get(x, cluster))
    if op=="median":
        return df.groupby(df.columns, axis=1).median()
    if op=="mean":
        return df.groupby(df.columns, axis=1).mean()
    if op=="sum":
        return df.groupby(df.columns, axis=1).sum()
    else:
        return "No se que funcion me has dado"



StartDate = '2020-5-06 00:48'
EndDate = '2020-6-06 00:50'
Value='DockBikes'
Comparator = 'd'
Accuracy = "h"
Interval=''




df = ChooseRange(data, StartDate, EndDate, Value)
ChooseComparison(df, Comparator, Accuracy )
# df = df.groupby([df.index.day, df.index.hour]).median()
Decode = Decoder('../DataFrames/DatosEstaticos.txt')

# ChooseInterval(df, 'd')
Clusterization(df, 'RegionName', 'sum', Decode)

ID,C,E,N,O,S
Date,,,,,
2020-05-06 01:26:07.581958,294.0,585.0,499.0,642.0,419.0
2020-05-06 02:26:08.586676,282.0,607.0,494.0,634.0,442.0
2020-05-06 03:26:10.610401,274.0,606.0,488.0,634.0,460.0
2020-05-06 04:26:13.764521,273.0,602.0,488.0,634.0,464.0
2020-05-06 05:26:15.421163,274.0,611.0,491.0,642.0,473.0
...,...,...,...,...,...
2020-06-05 20:48:02.338223,364.0,314.0,265.0,574.0,168.0
2020-06-05 21:48:02.498090,346.0,352.0,296.0,632.0,185.0
2020-06-05 22:48:04.242737,334.0,374.0,348.0,679.0,298.0


# class Ploteo

Para obtener datos finales y que se pueda meter facilmente en las graficas. Solo funciona para unafecha determinada, no puedes meter una tabla entera.

In [3]:
toPlot = Stations.Ploteo(df.iloc[0], Decode)
toPlot.Value
toPlot.Latitude
toPlot.Longitude
toPlot.Address
toPlot.Date
toPlot.Index

# Decoder Info

In [13]:
Decode = Decoder('../DataFrames/DatosEstaticos.txt')
Decode.infoStation(['Lavapiés'])

In [38]:
Decode._obj['Latitude'].loc[data.iloc[0]['DockBikes'].dropna().index]


ID
1      40.4168961
2      40.4170009
3      40.4205886
4      40.4302937
5      40.4285524
          ...    
200    40.4318611
201    40.4324722
207    40.4343611
211      40.44175
213       40.4535
Name: Latitude, Length: 184, dtype: object

In [36]:
data.iloc[0].dropna().index.get_level_values(1)

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            184, 185, 189, 190, 191, 200, 201, 207, 211, 213],
           dtype='int64', name='ID', length=1656)

In [37]:
data.iloc[0]['DockBikes'].dropna().index

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            184, 185, 189, 190, 191, 200, 201, 207, 211, 213],
           dtype='int64', name='ID', length=184)